In [1]:
from gurobipy import Model, GRB, quicksum

# Set
I = [1, 2, 3]  # Nodes with the highest demand
S = [1, 2]  # Vehicles
R = [1, 2]  # Routines
H = [0] + I  # Nodes + depot

#Parameter
d = {(i, j): ... for i in H for j in H}  # Distance
Qi = {i: ... for i in I}  # Demand
FC = ...  # Fixed cost
OC = ...  # Operational cost
P = ...  # Maximumncapacity
M = 100000  # Big M

# Model
m = Model("scooter_collection")

# Decision variables
X = m.addVars(I, I, S, R, vtype=GRB.BINARY, name="X")
T = m.addVars(I, S, R, vtype=GRB.INTEGER, name="T")
D = m.addVars(I, S, R, vtype=GRB.INTEGER, name="D")
y = m.addVars(I, vtype=GRB.INTEGER, name="y")
N = m.addVars(H, S, R, vtype=GRB.INTEGER, name="N")

# Objective function
m.setObjective(quicksum(FC * X[0, j, s, 1] for j in I for s in S) +
               quicksum(OC * d[i, j] * X[i, j, s, r] for i in H for j in H for s in S for r in R), GRB.MINIMIZE)

# Constraints
m.addConstrs(quicksum(T[i, s, r] for s in S for r in R) == Qi[i] for i in I)
m.addConstrs(quicksum(X[i, j, s, r] for i in I) <= 1 for j in I for s in S for r in R)
m.addConstrs(quicksum(X[i, j, s, r] for j in I) <= 1 for i in I for s in S for r in R)
m.addConstrs(T[i, s, r] <= M * quicksum(X[j, i, s, r] for j in H) for i in I for s in S for r in R)
m.addConstrs(quicksum(X[0, j, s, r] for j in I) <= 1 for s in S for r in R)
m.addConstrs(quicksum(X[i, 0, s, r] for i in I) <= 1 for s in S for r in R)
m.addConstrs(X[0, j, s, r+1] <= X[0, j, s, r] for j in I for s in S for r in range(len(R)-1))
m.addConstr(quicksum(D[1, s, r] for r in R for s in S) == 8)
m.addConstr(quicksum(D[2, s, r] for r in R for s in S) == 7)
m.addConstr(quicksum(D[3, s, r] for r in R for s in S) == 5)
m.addConstrs(D[i, s, r] <= M * quicksum(X[j, i, s, r] for j in H) for i in I for s in S for r in R)
m.addConstrs(N[0, s, r] == quicksum(D[i, s, r] for i in I) for s in S for r in R)
m.addConstrs(N[i, s, r] + T[j, s, r] - D[j, s, r] - N[j, s, r] <= M * (1 - X[i, j, s, r]) for i in H for j in I for s in S for r in R)
m.addConstrs(N[i, s, r] <= P for i in H for s in S for r in R)
m.addConstrs(y[i] - y[j] + (1 + len(I)) * X[i, j, s, r] <= len(I) for i in I for j in I if i != j for s in S for r in R)

# Optimize model
m.optimize()


Academic license - for non-commercial use only - expires 2025-06-06
Using license file C:\Users\wlsru\gurobi.lic


KeyError: (0, 1, 1, 1)